# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---

## Step 1: Dataset Exploration

Visualize the German Traffic Signs Dataset. This is open ended, some suggestions include: plotting traffic signs images, plotting the count of each sign, etc. Be creative!


The pickled data is a dictionary with 4 key/value pairs:

- features -> the images pixel values, (width, height, channels)
- labels -> the label of the traffic sign
- sizes -> the original width and height of the image, (width, height)
- coords -> coordinates of a bounding box around the sign in the image, (x1, y1, x2, y2). Based the original image (not the resized version).

In [3]:
# Load pickled data
import pickle
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

%matplotlib inline

# TODO: fill this in based on where you saved the training and testing data
training_file = '/Users/safdar/Datasets/self-driving-car/traffic-signs-data/train.p'
testing_file = '/Users/safdar/Datasets/self-driving-car/traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
x_train, y_train, sizes_train, coords_train = train['features'], train['labels'], train['sizes'], train['coords']
x_test, y_test, sizes_test, coords_test = test['features'], test['labels'], train['sizes'], train['coords']

print (train.keys())

dict_keys(['coords', 'labels', 'sizes', 'features'])


In [7]:
# Visualization tool
def draw_with_box(i):
    img = Image.fromarray(x_train[i], 'RGB')

    # Original dimensions of the image
    org_x = sizes_train[i][0]
    org_y = sizes_train[i][1]
    
    # Scaling : Image has been scaled from its original size down to the processing size (32)
    sz_x = len(x_train[i])
    sz_y = len(x_train[i][0])
    
    # Ratio : Of actual/original (this is to scale the rectangular coordinates around image)
    rat_x = float(sz_x / org_x)
    rat_y = float(sz_y / org_y)
    
    # Rectangle: Scale the rectangular box based on the scaling above
    x1 = rat_x * coords_train[i][0]
    x2 = rat_x * coords_train[i][2]
    y1 = rat_y * coords_train[i][1]
    y2 = rat_y * coords_train[i][3]
    
    # Logging
    print ("Img : ", sz_x, " x ", sz_y)
    print ("Orig: ", org_x, " x ", org_y)
    print ("Box : [", x1, y1, x2, y2, "]")
    
    #img = Image.fromarray(x_train[i], 'RGB')
    #img.show()
    
    # Title:
    title = "Class: {}".format(y_train[i])

    # Plot image and rectangular box
    plt.imshow(x_train[i][:][:][:])
    plt.plot([x1, x1], [y1, y2], 'k-')
    plt.plot([x2, x2], [y1, y2], 'k-')
    plt.plot([x1, x2], [y1, y1], 'k-')
    plt.plot([x1, x2], [y2, y2], 'k-')
    plt.title(title)

    plt.figure()
    
#def extract_and_resize (img, coords):
    
    
def classcount ():
    mx = 0
    for cls in y_train:
        if cls > mx:
            mx = cls
            
    return mx

In [5]:
### To start off let's do a basic data summary.

# Number of training examples
n_train = len(x_train)

# Number of testing examples
n_test = len (x_test)

# What's the shape of an image?
image_shape = len(x_train[0][:][:][:]) # Images are 32 x 32

# How many channels?
n_channels = len(x_train[0][0][0])

# How many classes are in the dataset
n_classes = classcount()

# Image size, when flattened:
img_size_flat = image_shape * image_shape * n_channels

print("Number of training examples =", n_train)
print("Image dimensions = {} x {}".format(image_shape, image_shape))
print("Number of channels: ", n_channels)
print("Flattened image length: ", img_size_flat)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 39209
Image dimensions = 32 x 32
Number of channels:  3
Flattened image length:  3072
Number of testing examples = 12630
Image data shape = 32
Number of classes = 42


In [6]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
draw_with_box(34888)
draw_with_box(18734)
draw_with_box(1)

ValueError: too many values to unpack (expected 2)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Your model can be derived from a deep feedforward net or a deep convolutional network.
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [9]:
### Preprocess the data here.

# Preprocessing steps - Training Data:
# - Convert to grayscale
# - Extract the coordinates from the image
# - Resize to 32 x 32
# - Normalize values between 0.0 and 0.1
# - Flatten to a 2-D array [None, img_size_flat]

# Prepare expected outputs:
# - One-Hot encode the y_train labels for a softmax network
# - One-Hot encode the y_test labels in the same way

# One-Hot encode the y_train labels vector
y_train_hot = np.zeros((len(y_train), y_train.max()+1), np.int)
y_train_hot[np.arange(len(y_train)), y_train] = 1

In [10]:
import tensorflow as tf

#
#
# Regular Neural Network
#
#

x = tf.placeholder(tf.float32, [None, image_shape, image_shape, n_channels])
y_true_cls = tf.placeholder(tf.int64, [None])
y_true_hot = tf.placeholder(tf.int64, [None, n_classes])

weights = tf.Variable(tf.zeros([img_size_flat, n_classes]))
biases = tf.Variable(tf.zeros([n_classes]))

logits = tf.matmul(x, weights) + biases
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred, dimension=1)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(cost)

correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


ValueError: Shape must be rank 2 but is rank 4

In [43]:
# Training:
def optimize_batch (session):
    feed_dict_train = {x: x_train, y_true: y_train_1_hot}
    session.run(optimizer, feed_dict = feed_dict_train)

with tf.Session() as session:
    session.run(tf.initialize_all_variables())
    optimize_batch(session)


ValueError: Cannot feed value of shape (39209, 32, 32, 3) for Tensor 'Placeholder_51:0', which has shape '(?, 1024)'

In [32]:
# Testing
feed_dict_test = {x: x_test, y_true_cls: y_test, y_true_cls: y_true} # Setup y_true
def print_accuracy(feed_dict, datatype):
    acc = session.run(accuracy, feed_dict = feed_dict)
    print ("Accuracy on {} set: {0:.1%}".format(datatype, acc))

In [ ]:
def optimize(num_iterations):
    for i in range(num_iterations):
        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
        x_batch, y_true_batch = data.train.next_batch(batch_size)
        
        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        # Note that the placeholder for y_true_cls is not set
        # because it is not used during training.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}

        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict=feed_dict_train)

In [37]:
def plot_weights():
    # Get the values for the weights from the TensorFlow variable.
    w = session.run(weights)
    
    # Get the lowest and highest values for the weights.
    # This is used to correct the colour intensity across
    # the images so they can be compared with each other.
    w_min = np.min(w)
    w_max = np.max(w)

    # Create figure with 3x4 sub-plots,
    # where the last 2 sub-plots are unused.
    fig, axes = plt.subplots(3, 4)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Only use the weights for the first 10 sub-plots.
        if i<10:
            # Get the weights for the i'th digit and reshape it.
            # Note that w.shape == (img_size_flat, 10)
            image = w[:, i].reshape(img_shape)

            # Set the label for the sub-plot.
            ax.set_xlabel("Weights: {0}".format(i))

            # Plot the image.
            ax.imshow(image, vmin=w_min, vmax=w_max, cmap='seismic')

        # Remove ticks from each sub-plot.
        ax.set_xticks([])
        ax.set_yticks([])


In [35]:
def plot_images(images, cls_true, cls_pred=None):
    assert len(images) == len(cls_true) == 9
    
    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Plot image.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
    

In [ ]:
def print_confusion_matrix():
    # Get the true classifications for the test-set.
    cls_true = data.test.cls
    
    # Get the predicted classifications for the test-set.
    cls_pred = session.run(y_pred_cls, feed_dict=feed_dict_test)

    # Get the confusion matrix using sklearn.
    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)

    # Print the confusion matrix as text.
    print(cm)

    # Plot the confusion matrix as an image.
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)

    # Make various adjustments to the plot.
    plt.tight_layout()
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')

In [36]:
def plot_example_errors():
    # Use TensorFlow to get a list of boolean values
    # whether each test-image has been correctly classified,
    # and a list for the predicted class of each image.
    correct, cls_pred = session.run([correct_prediction, y_pred_cls],
                                    feed_dict=feed_dict_test)

    # Negate the boolean array.
    incorrect = (correct == False)
    
    # Get the images from the test-set that have been
    # incorrectly classified.
    images = x_test[incorrect]
    
    # Get the predicted classes for those images.
    cls_pred = cls_pred[incorrect]

    # Get the true classes for those images.
    cls_true = data.test.cls[incorrect]
    
    # Plot the first 9 images.
    plot_images(images=images[0:9],
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])


### Question 1 

_Describe the techniques used to preprocess the data._

**Answer:**

In [2]:
### Generate data additional (if you want to!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

### Question 2

_Describe how you set up the training, validation and testing data for your model. If you generated additional data, why?_

**Answer:**

In [6]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [5]:
### Train your model here.
### Feel free to use as many code cells as needed.

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem?_

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [3]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It would be helpful to plot the images in the notebook._



**Answer:**

In [4]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the dataset?_


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.11/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*


**Answer:**

### Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.